In [1]:
import sys; sys.path.append("../")

import pandas as pd

from lib.explore import show_unique
from lib.match import ThresholdMatcher, JaroWinklerSimilarity, ColumnsIndex, StringSimilarity
from clean.baton_rouge_so_cprr import clean18

In [2]:
df = clean18()
df

../clean/baton_rouge_so_cprr.py:13: FutureWarning: The default value of regex will change from True to False in a future version.
  names1 = df.name.str.strip().str.replace(
../lib/clean.py:152: FutureWarning: The default value of regex will change from True to False in a future version.
  return series.str.strip().str.replace(r"[^\w-]+", " ").str.replace(r"\s+", " ")\
../clean/baton_rouge_so_cprr.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, 'charges'] = infractions.iloc[:, 1].fillna('').str.strip()\
../lib/clean.py:128: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.strip().str.lower()\
../lib/clean.py:120: FutureWarning: The default value of regex will change from True to False in a future version.
  df.loc[:, col] = df[col].str.strip().str.lower()\
../clean/baton_rouge_so_cprr.py:24: FutureWarning: The default value of regex will change fr

,badge_no,rank_desc,race,sex,birth_year,complainant_type,disposition,action,department_desc,first_name,...,rank_month,rank_day,occur_year,occur_month,occur_day,occur_time,data_production_year,agency,uid,complaint_uid
0,2206,sergeant,white,male,1984,public,founded,forwarded completed report to administration f...,uniform patrol,william,...,2,6,2018,1,26,21:00,2018,Baton Rouge SO,402619644d9eff018cd39a757bcc5153,3be78cbff1b1fc9190b766736bc43d89
1,1711,corporal,white,male,1984,administration,founded,7 day suspension; 6 month loss of take-home ve...,scat,eric,...,2,13,2018,2,20,00:00,2018,Baton Rouge SO,7b22c2ab899789ff98f5cd884b4175be,e976d7fc0980ccbf7cbe7681ab92cf6d
2,1308,lieutenant,white,male,1968,coworker,founded,resigned,parish prison (training),eddie,...,9,6,2018,3,12,00:00,2018,Baton Rouge SO,1f86bca13a125c7422d192a14159ac21,385588d4a6151b0a8bfc3979d2336918
3,1534,sergeant,black,male,1966,coworker,inconclusive,none,construction services,troy,...,6,15,2018,3,15,00:00,2018,Baton Rouge SO,93c7b5cba8c235cb5aa61b013922d71a,39b730b51a07c87cfa710b782ddc59ee
4,2527,deputy,black,female,1986,public,founded,verbal warning from captain flynn about courte...,community policing,pashion,...,12,16,2018,4,8,15:30,2018,Baton Rouge SO,6cf20697312e7736147155175876da95,0843cadef1bc4afcff1e6969a527767d
5,1797,corporal,white,female,1982,supervisor,founded,none,financial crimes,leighann,...,2,9,2018,4,11,10:00,2018,Baton Rouge SO,b91b655b93a3c0de5c46674b3472867f,d4d33262b15d451e8b664c8e3d8c4336
6,769,captain,black,female,1965,subordinate,unfounded,none,communications,eleanor,...,6,16,2018,4,13,12:00,2018,Baton Rouge SO,d4abd7442439b47d115132746ecdf7da,841b429998df6000c222800fcdb144b2
7,2067,deputy,white,male,1982,public,inconclusive,none,scat,john,...,11,17,2018,4,19,00:00,2018,Baton Rouge SO,b180787c744ce3e7383aa683f40b51ea,66ad764eabb35b47b2a5d18014932ec9
8,1519,lieutenant,white,male,1982,administration,founded,5 day suspension,uniform patrol,justin,...,11,27,2018,4,22,00:00,2018,Baton Rouge SO,3f6d99799d87868c8b9f69126f4b331b,3451d48ee15eaad7a4f33c8bb25ba301
9,2363,sergeant,white,male,1983,public,unfounded,none,uniform patrol,dustin,...,11,27,2018,5,10,00:00,2018,Baton Rouge SO,93efca0952d59008ceb2f8f0baaeb7ac,321d49855632efb7eeb08697c1813366


In [4]:
pprr = pd.read_csv("../data/match/pprr_baton_rouge_csd_2019.csv")
pprr

,year,last_name,first_name,middle_initial,department_code,department_desc,rank_code,rank_desc,employment_status,annual_salary,...,hire_year,hire_month,hire_day,term_year,term_month,term_day,agency,uid,level_1_cert_date,last_pc_12_qualification_date
0,2018,williams,decory,l,5040,police-operational serv bureau,2070,senior stock clerk,active,26541.06,...,2014,6,23,NaN,NaN,NaN,Baton Rouge CSD,6b1b11e685bf2b6be76eb85a56ff47a6,NaN,NaN
1,2018,weissberg,deena,j,5020,police-uniform patrol bureau,5005,police officer,active,53024.53,...,2010,3,31,NaN,NaN,NaN,Baton Rouge CSD,830b637f0eff1672df622c228005f8ed,NaN,NaN
2,2018,graham,michael,h,5050,police-communications,5115,police comm officer i-42 hrs,inactive,54243.63,...,2007,4,21,2019.0,1.0,22.0,Baton Rouge CSD,e91081834424eab42d2c1519c93c558a,NaN,NaN
3,2018,gillich,joshua,m,5070,police-criminal investigation,5005,police officer,active,28738.00,...,2008,12,8,NaN,NaN,NaN,Baton Rouge CSD,a2f8eb18d214233bc8cc61d1eec86653,2009-04-22,2020-07-15
4,2018,saleh,haled,NaN,5020,police-uniform patrol bureau,5005,police officer,inactive,62144.25,...,2006,6,12,2019.0,4.0,26.0,Baton Rouge CSD,38ba903c314b093eae2b3dac8c4587ee,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1957,2018,howard,walter,c,5060,police-special operations,6525,school crossing guard,active,942.50,...,2018,10,24,NaN,NaN,NaN,Baton Rouge CSD,7853fef8a85f2e467609190b8d430b19,NaN,NaN
1958,2018,houston,stacy,m,5060,police-special operations,6525,school crossing guard,active,928.00,...,2018,10,24,NaN,NaN,NaN,Baton Rouge CSD,2365ddfa5da9a0d53954985f350d374f,NaN,NaN
1959,2018,dyson,kenneth,w,5060,police-special operations,6525,school crossing guard,active,928.00,...,2018,10,24,NaN,NaN,NaN,Baton Rouge CSD,3fc29868f45c3b698372ee18767e407c,NaN,NaN
1960,2018,augustus,robert,l,5060,police-special operations,6525,school crossing guard,inactive,913.50,...,2018,10,24,2019.0,6.0,10.0,Baton Rouge CSD,6a339e98b96903c875416d0516c3780a,NaN,NaN


In [5]:
dfa = df[['first_name', 'middle_initial', 'last_name', 'uid']]\
    .drop_duplicates().set_index('uid')
dfa.loc[:, 'fnc'] = dfa.first_name.map(lambda x: x[:1])
dfb = pprr[['first_name', 'middle_initial', 'last_name', 'uid']]\
    .drop_duplicates().set_index('uid')
dfb.loc[:, 'fnc'] = dfb.first_name.map(lambda x: x[:1])
matcher = ThresholdMatcher(dfa, dfb, ColumnsIndex(['fnc']), {
    'middle_initial': StringSimilarity(),
    'first_name': JaroWinklerSimilarity(),
    'last_name': JaroWinklerSimilarity(),
})

In [6]:
matcher.get_sample_pairs()

first_name  \
score_range pair_idx sim_score row_key                                       
0.90-0.85   0        0.855868  3ab64d2e2b554256b897419613449ce9    kenneth   
                               0bbcbb4e4b2dc172244df569f88263ef    kenneth   
            1        0.854506  3ab64d2e2b554256b897419613449ce9    kenneth   
                               6caac21bb605a9b1c43a753027bca7f7    kenneth   
0.85-0.80   0        0.840829  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               fb71af1938a6ea981e50b2309d488127      duane   
0.80-0.75   0        0.779257  3ab64d2e2b554256b897419613449ce9    kenneth   
                               2d105194b7900c781a1d801c707c7a30      kevin   
            1        0.777897  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               01956d97bd4fda9ae864d16108c11e34      devin   
            2        0.776899  3ab64d2e2b554256b897419613449ce9    kenneth   
                               8739ece76fdf81f712c67fee74af103e      kynon   
            3        0.753519  5d99e8da3c3ab0a8c67482d5b5aa708e   courtney   
                               73af887d940cce5865dbab8e49fccbcf    charles   
0.75-0.70   0        0.718280  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               54ea3b30308cbb9257d8222f70c50739     darren   
            1        0.718280  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               c59a7054419b1c7a0d552c92b8bb1d4a     demika   
            2        0.718036  3ab64d2e2b554256b897419613449ce9    kenneth   
                               b1e3c2aafbe022899ca19fe5b6451e62       kama   
            3        0.708083  93efca0952d59008ceb2f8f0baaeb7ac     dustin   
                               d716dc3b5921595dc992f40a72b2c09d    deborah   
            4        0.704746  3ab64d2e2b554256b897419613449ce9    kenneth   
                               c4994d8353864109fca8fe4c881bf20f    kenneth   

                                                                middle_initial  \
score_range pair_idx sim_score row_key                                           
0.90-0.85   0        0.855868  3ab64d2e2b554256b897419613449ce9              j   
                               0bbcbb4e4b2dc172244df569f88263ef              j   
            1        0.854506  3ab64d2e2b554256b897419613449ce9              j   
                               6caac21bb605a9b1c43a753027bca7f7              j   
0.85-0.80   0        0.840829  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               fb71af1938a6ea981e50b2309d488127              l   
0.80-0.75   0        0.779257  3ab64d2e2b554256b897419613449ce9              j   
                               2d105194b7900c781a1d801c707c7a30              j   
            1        0.777897  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               01956d97bd4fda9ae864d16108c11e34              l   
            2        0.776899  3ab64d2e2b554256b897419613449ce9              j   
                               8739ece76fdf81f712c67fee74af103e              j   
            3        0.753519  5d99e8da3c3ab0a8c67482d5b5aa708e              r   
                               73af887d940cce5865dbab8e49fccbcf              r   
0.75-0.70   0        0.718280  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               54ea3b30308cbb9257d8222f70c50739              l   
            1        0.718280  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               c59a7054419b1c7a0d552c92b8bb1d4a              l   
            2        0.718036  3ab64d2e2b554256b897419613449ce9              j   
                               b1e3c2aafbe022899ca19fe5b6451e62              j   
            3        0.708083  93efca0952d59008ceb2f8f0baaeb7ac              l   
                               d716dc3b5921595dc992f40a72b2c09d              l   
            4        0.704746  3ab64d2e2b554256b897419613449ce9    